In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
cover = cv2.imread(r"C:\Users\varsha\Pictures\cover_3.jpg")
wrap = cv2.imread(r"C:\Users\varsha\Pictures\wrap.jpg")

# convert to RGB
cover_rgb = cv2.cvtColor(cover, cv2.COLOR_BGR2RGB)
wrap_rgb = cv2.cvtColor(wrap, cv2.COLOR_BGR2RGB)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 10))
ax[0].imshow(cover_rgb)
ax[1].imshow(wrap_rgb)

In [ ]:
# conver to gray scale
cover_gray = cv2.cvtColor(cover_rgb, cv2.COLOR_RGB2GRAY)
wrap_gray = cv2.cvtColor(wrap_rgb, cv2.COLOR_RGB2GRAY)

fig, ax = plt.subplots(1, 2, figsize = (10, 10))
ax[0].imshow(cover_gray, cmap = 'gray')
ax[1].imshow(wrap_gray, cmap = 'gray')

In [ ]:
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

# detect ORB features and compute descriptor 
orb = cv2.ORB_create(MAX_FEATURES)

keypt_1, descriptor_1 = orb.detectAndCompute(cover_gray, None)
keypt_2, descriptor_2 = orb.detectAndCompute(wrap_gray, None)

# display
img_1 = cv2.drawKeypoints(cover_rgb, keypt_1, outImage = np.array([]), color =(255, 0, 0), flags =cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
img_2 = cv2.drawKeypoints(wrap_rgb, keypt_2, outImage = np.array([]), color =(255, 0, 0), flags =cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)

fig, ax = plt.subplots(1, 2, figsize = (10, 10))
ax[0].imshow(img_1)
ax[1].imshow(img_2)

In [ ]:
# Match features.
matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
matches = matcher.match(descriptor_1, descriptor_2, None)


# # sort matches by score
sorted(matches, key = lambda x: x.distance, reverse = False)


# remove not so good matches
goodMatches = int(len(matches) * 0.1)

matches = matches[:goodMatches]

# display
img_1 = cv2.drawMatches(cover_rgb, keypt_1, wrap_rgb, keypt_2, matches, None)

plt.imshow(img_1)


In [ ]:
# Extract location of goodMatches
pts1 = np.zeros((len(matches), 2), dtype = np.float32)
pts2 = np.zeros((len(matches), 2), dtype = np.float32)

for i, match in enumerate(matches):
    pts1[i, :] = keypt_1[match.queryIdx].pt
    pts2[i, :] = keypt_2[match.trainIdx].pt
    
    
# find homography
h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC)

# use homography
height, width, channels = wrap_rgb.shape
cover_R = cv2.warpPerspective(cover_rgb, h, (width, height))


fig, ax = plt.subplots(1, 2, figsize = (10, 10))
ax[0].imshow(cover_rgb)
ax[1].imshow(cover_R)

In [ ]:
# Initialize ORB detector
orb = cv2.ORB_create()

# Detect keypoints and compute descriptors
keypoints_1, descriptors_1 = orb.detectAndCompute(cover_gray, None)
keypoints_2, descriptors_2 = orb.detectAndCompute(wrap_gray, None)

# Match descriptors using a brute-force matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(descriptors_1, descriptors_2)

# Sort matches by distance
matches = sorted(matches, key=lambda x: x.distance)

# Apply ratio test to filter good matches
good_matches = []
for m in matches:
    if m.distance < 0.75 * matches[0].distance:
        good_matches.append(m)

print(good_matches)
# matches = matches[:goodMatches]

# # Extract location of goodMatches
# pts1 = np.zeros((len(goodMatches), 2), dtype = np.float32)
# pts2 = np.zeros((len(goodMatches), 2), dtype = np.float32)

# for i, match in enumerate(goodMatches):
#     pts1[i, :] = keypt_1[match.queryIdx].pt
#     pts2[i, :] = keypt_2[match.trainIdx].pt
    
    
# # find homography
# h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC)

# # use homography
# height, width, channels = wrap_rgb.shape
# cover_R = cv2.warpPerspective(cover_rgb, h, (width, height))


# fig, ax = plt.subplots(1, 2, figsize = (10, 10))
# ax[0].imshow(cover_rgb)
# ax[1].imshow(cover_R)

# Draw top matches
image_matches = cv2.drawMatches(cover_rgb, keypoints_1, wrap_rgb, keypoints_2, good_matches, None)

# Show image with matches
plt.imshow(image_matches)


In [ ]:
# display
img_1 = cv2.drawMatches(cover_rgb, keypt_1, wrap_rgb, keypt_2, matches, None)

plt.imshow(img_1)